In [ ]:
# Import the libraries
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
from odc.stac import configure_rio, stac_load
import dask.distributed
import dask.utils
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Set up Dask client for parallel processing
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)

# Configure rio with dynamic resolution
resolution = 20
memory_limit = dask.utils.parse_bytes(client.cluster.workers[0].memory_manager.memory_limit)
SHRINK = 4
if memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # Adjust chunk size if memory is limited

resolution = resolution * SHRINK


In [ ]:
# Define the area of interest (AOI) for Lake Michigan
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-88.2, 43.0],  # Lower-left corner
            [-86.1, 43.0],  # Lower-right corner
            [-86.1, 45.0],  # Upper-right corner
            [-88.2, 45.0],  # Upper-left corner
            [-88.2, 43.0],  # Closing the polygon
        ]
    ],
}

#  time span of 3 months
time_of_interest = "2023-06-01/2023-12-01"

# Query the catalog for the data
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest
)
items = list(search.items())
print(f"Returned {len(items)} Items")


In [ ]:
# Load the data from the catalog with dynamic chunking and handle missing data
xx = stac_load(
    items,
    chunks={"x": 1024 * SHRINK, "y": 1024 * SHRINK},  # Dynamically adjust chunk size
    patch_url=planetary_computer.sign,
    resolution=resolution,
    dtype="uint16",  # Handle missing data by marking nodata values
    nodata=0
)

# Display loaded data
print(f"Bands: {','.join(list(xx.data_vars))}")
display(xx)


In [ ]:
# Function to convert data to float and handle missing nodata values
def to_float(xx, nodata_value=None):
    _xx = xx.astype("float32")  # Convert data to float32 for precision
    if nodata_value is None:
        nodata_value = _xx.attrs.pop("nodata", None)  # Fetch nodata value if exists
    if nodata_value is not None:
        return _xx.where(xx != nodata_value)  # Replace nodata with NaN
    return _xx

# Convert specific bands to float32 and handle missing data
b05 = to_float(xx.B05)  # Red-Edge band
b04 = to_float(xx.B04)  # Red band


In [ ]:
# Calculate NDCI with small constant to avoid division by zero
ndci = (b05 - b04) / (b05 + b04 + 1e-6)

# Display the calculated NDCI
display(ndci)

In [ ]:
ndci = ndci.fillna(ndci.mean())

In [ ]:
chl = 826.57*(ndci**3) - 176.43*(ndci**2) + 19*(ndci) + 4.071
display(chl)

In [ ]:
# Select a subset of coordinates (first 50 coordinates)
chl_subset = chl[:, :50, :50]

In [ ]:
print(chl_subset.shape)

In [ ]:
# Define a function to prepare the time series data for LSTM
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Set time steps (e.g., using last 5 values to predict the next)
time_steps = 10
#forecast_horizon = 5  # How many future timesteps to predict
X, y = create_dataset(chl_subset, time_steps)
print(f"X shape: {X.shape}, y shape: {y.shape}")



In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
# Reshape the input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], chl_subset.shape[1] * chl_subset.shape[2]))
y = np.reshape(y, (y.shape[0], chl_subset.shape[1] * chl_subset.shape[2]))

print(f"X shape: {X.shape}, y shape: {y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Normalize input features
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)

X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# Normalize target variable
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Reshape back to the original shape
y_train_scaled = y_train_scaled.reshape(y_train.shape)  
y_test_scaled = y_test_scaled.reshape(y_test.shape) 

In [ ]:
print(X_train_scaled.shape, y_train_scaled.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(y.shape[1], activation='linear')
])
# Output a single value for each spatial location (the predicted next time point)
model.compile(optimizer='adam', loss='mean_squared_error')


# Print the model summary
model.summary()

In [ ]:
print(X_train.shape)
print(y_train.shape)
# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
results = model.evaluate(X_test_scaled, y_test_scaled)
print(f"Model loss (MSE): {results}")
print("RMSE: ", np.sqrt(results))

# Make predictions
predictions = model.predict(X_test_scaled)
print(predictions)

In [ ]:
print(predictions.shape)

In [ ]:
print(y_test_scaled.min(), y_test_scaled.max())

In [ ]:
print(predictions.min(), predictions.max())

In [ ]:
y_pred = scaler_y.inverse_transform(predictions.reshape(-1, 1))

# Reshape the predictions back to the original shape
y_pred = y_pred.reshape(y_test.shape)

In [ ]:
print(y_pred.min(), y_pred.max())

In [ ]:
print(y_pred.shape)

In [ ]:
print(y_pred)
print(y_test)

In [ ]:
# import numpy as np

# # Save predictions as a Numpy .npy file
# np.save("predictions.npy", y_pred)
# np.save("y_test.npy", y_test)

In [ ]:
# Reshape y_test_inverse back to its original shape
y_test_inverse = y_test.reshape((y_test.shape[0], chl_subset.shape[1], chl_subset.shape[2]))
print(y_test_inverse)

In [ ]:
print(y_test_inverse.shape)

In [ ]:
# Reshape y_test_inverse back to its original shape
y_pred_inverse = y_pred.reshape((y_pred.shape[0], chl_subset.shape[1], chl_subset.shape[2]))
print(y_pred_inverse.shape)

In [ ]:
# Select a timestep to visualize (e.g., the first timestep in the test set)
timestep = 0

# Extract the data for the selected timestep
y_pred_timestep = y_pred_inverse[timestep]

# Extract the actual coordinates
x_coords = chl_subset.coords['x'].values
y_coords = chl_subset.coords['y'].values

# Create the visualization
plt.figure(figsize=(8, 6))
plt.pcolormesh(x_coords, y_coords, y_pred_timestep, cmap='viridis', shading='auto')
plt.colorbar(label='Chlorophyll Value')
plt.title(f'Predicted Chlorophyll Values at Timestep {timestep}')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

In [ ]:
# Select a specific grid point (e.g., x_idx = 0, y_idx = 0 for top-left corner)
x_idx, y_idx = 0, 0

# Extract chlorophyll values over time for the specific point
time_series = y_pred_inverse[:, y_idx, x_idx]

# Plot chlorophyll values over time
plt.figure(figsize=(10, 6))
plt.plot(range(len(time_series)), time_series, marker='o', linestyle='-', label=f'Point ({x_idx}, {y_idx})')
plt.title('Chlorophyll Value Over Time at Specific Grid Point')
plt.xlabel('Time Step')
plt.ylabel('Chlorophyll Value')
plt.grid()
plt.legend()
plt.show()


In [ ]:
client.close()

In [ ]:
model.save('LSTM_model.h5')
print("Model saved to LSTM_model.h5")

In [ ]:
# Save the current_input array to a .npy file
np.save('test_input.npy', X_test_scaled )
print("current_input saved to test_input.npy")

In [ ]:
# Save the y_train array to a .npy file
np.save('y_train.npy', y_train)
print("y_train saved to y_train.npy")